In [ ]:
! pip install "tensorflow-text>=2.11"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.8/489.8 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 75.9 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.15.0
    Uninstalling wrapt-1.15.0:
      Successfully uninstalled wrapt-1.15.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.13.0
    Uninstalling tensorflow-estimator-2.13.0:
      Successfully uninstalled tensorflow-estimator-2.13.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes

In [ ]:
import numpy as np
import pathlib
import os
import tensorflow_text as tf_text
import tensorflow as tf
from tensorflow import keras
from keras import layers

In [ ]:
path_to_zip = keras.utils.get_file('spa-eng.zip',origin = 'http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip',extract = True)
path_to_file = pathlib.Path(path_to_zip).parent/'spa-eng'/'spa.txt'

2638744/2638744 [==============================] - 0s 0us/step


In [ ]:
def load_data(path):
  text = path.read_text(encoding='utf-8')

  lines = text.splitlines()
  pairs = [line.split('\t') for line in lines]

  context = np.array([context for target,context in pairs])
  target = np.array([target for target,context in pairs])

  return target, context

In [ ]:
target_raw,context_raw = load_data(path_to_file)

print(context_raw[1])
print(target_raw[1])

Vete.
Go.


In [ ]:
Buffer_Size = len(context_raw)
Batch_Size = 64

is_train = np.random.uniform(size=(len(target_raw),)) < 0.8

train_raw = (tf.data.Dataset
    .from_tensor_slices((context_raw[is_train], target_raw[is_train])).shuffle(Buffer_Size).batch(Batch_Size))
val_raw = (tf.data.Dataset
    .from_tensor_slices((context_raw[~is_train], target_raw[~is_train])).shuffle(Buffer_Size).batch(Batch_Size))

In [ ]:
for example_context_strings, example_target_strings in train_raw.take(1):
  print(example_context_strings[:5])
  print(example_target_strings[:5])
  break

tf.Tensor(
[b'Tom ya no lo necesitaba.' b'Tan solo d\xc3\xadganle la verdad.'
 b'No pens\xc3\xa9 que Tom quisiera volver a verme.'
 b'No se puede comprar en ning\xc3\xban otro lugar, solo all\xc3\xad.'
 b'Tom salt\xc3\xb3 al agua helada.'], shape=(5,), dtype=string)

tf.Tensor(
[b"Tom didn't need it anymore." b'Just tell him the truth.'
 b"I didn't think Tom would want to see me again."
 b"You can't buy it anywhere but there."
 b'Tom jumped into the cold water.'], shape=(5,), dtype=string)


In [ ]:
# Standarization
def tf_lower_and_split_punct(text):
  text = tf_text.normalize_utf8(text, 'NFKD')
  text = tf.strings.lower(text)
  text = tf.strings.regex_replace(text, '[^ a-z.?!,¿]', '')
  text = tf.strings.regex_replace(text, '[.?!,¿]', r' \0 ')
  text = tf.strings.strip(text)

  text = tf.strings.join(['[START]', text, '[END]'], separator=' ')
  return text

In [ ]:
# Now we do a text vectorization

max_vocab_size = 10000

context_text_processor = layers.TextVectorization(standardize=tf_lower_and_split_punct,
                                                  max_tokens=max_vocab_size,
                                                  ragged=True)

context_text_processor.adapt(train_raw.map(lambda context, target : context))

context_text_processor.get_vocabulary()[:15]

['',
 '[UNK]',
 '[START]',
 '[END]',
 '.',
 'que',
 'de',
 'el',
 'a',
 'no',
 'tom',
 'la',
 '?',
 '¿',
 'en']

In [ ]:
# text vectorization also for target values
target_text_processor = layers.TextVectorization(standardize=tf_lower_and_split_punct,
                                                  max_tokens=max_vocab_size,
                                                  ragged=True)

target_text_processor.adapt(train_raw.map(lambda context, target : target))

target_text_processor.get_vocabulary()[:15]


['',
 '[UNK]',
 '[START]',
 '[END]',
 '.',
 'the',
 'i',
 'to',
 'you',
 'tom',
 'a',
 '?',
 'is',
 'he',
 'in']

In [ ]:
context_vocab = np.array(context_text_processor.get_vocabulary())

In [ ]:
# process the dataset
def process_text(context , target):
  context = context_text_processor(context).to_tensor()
  target = target_text_processor(target)
  targ_in = target[:,:-1].to_tensor()
  targ_out = target[:,1:].to_tensor()
  return (context,targ_in),targ_out

train_ds = train_raw.map(process_text,tf.data.AUTOTUNE)
val_ds = val_raw.map(process_text, tf.data.AUTOTUNE)

In [ ]:
# Now build the encoder

class Encoder(layers.Layer):
  def __init__(self,text_processor,units):
    super(Encoder,self).__init__()
    self.text_processor = text_processor
    self.vocab_size = text_processor.vocabulary_size()
    self.units = units

    self.embedding = layers.Embedding(self.vocab_size,units,mask_zero = True)

    self.rnn = layers.Bidirectional(merge_mode = 'sum',
                                    layer = layers.GRU(units,return_sequences = True,recurrent_initializer='glorot_uniform'))

  def call(self,x):
    x = self.embedding(x)
    x = self.rnn(x)
    return x

  def convert_input(self,texts):
    texts = tf.convert_to_tensor(texts)
    if len(texts.shape) == 0:
      texts = tf.convert_to_tensor(texts)[tf.newaxis]
    context = self.text_processor(texts).to_tensor()
    context = self(context)
    return context

In [ ]:
# Now build the cross attention layer
UNITS = 256

class CrossAttention(layers.Layer):
  def __init__(self,units,**kwargs):
    super().__init__()
    self.mha = layers.MultiHeadAttention(key_dim=units,num_heads=1,**kwargs)
    self.layernorm = layers.LayerNormalization()
    self.add = layers.Add()

  def call(self,x,context):
    attn_output, attn_scores = self.mha(query=x,value=context,return_attention_scores=True)
    attn_scores = tf.reduce_mean(attn_scores,axis=1)
    self.last_attention_weights = attn_scores

    x = self.add([x,attn_output])
    x = self.layernorm(x)
    return x

In [ ]:
attention_layer = CrossAttention(UNITS)

embed = layers.Embedding(target_text_processor.vocabulary_size(),output_dim=UNITS,mask_zero=True)

In [ ]:
# Now we build the decoder

class Decoder(layers.Layer):
  @classmethod
  def add_method(cls,fun):
    setattr(cls,fun.__name__,fun)
    return fun

  def __init__(self,text_processor,units):
    super(Decoder,self).__init__()
    self.text_processor = text_processor
    self.vocab_size = text_processor.vocabulary_size()
    self.word_to_id = layers.StringLookup(vocabulary=text_processor.get_vocabulary(), mask_token='',oov_token='[UNK]')
    self.id_to_word = layers.StringLookup(vocabulary=text_processor.get_vocabulary(), mask_token='',oov_token='[UNK]',invert=True)
    self.start_token = self.word_to_id('[START]')
    self.end_token = self.word_to_id('[END]')
    self.units = units

    self.embedding = layers.Embedding(self.vocab_size,units,mask_zero=True)

    self.rnn = layers.GRU(units,return_sequences=True,return_state = True, recurrent_initializer='glorot_uniform')

    self.attention = CrossAttention(units)

    self.output_layer = layers.Dense(self.vocab_size)

In [ ]:
@Decoder.add_method
def call(self,context,x,state=None,return_state=False):
  x = self.embedding(x)
  x , state = self.rnn(x, initial_state = state)
  x = self.attention(x,context)
  self.last_attention_weights = self.attention.last_attention_weights
  logits = self.output_layer(x)
  if return_state:
    return logits, state
  else:
    return logits

decoder = Decoder(target_text_processor,UNITS)

In [ ]:
# Now build the model

class Translator(keras.Model):
  @classmethod
  def add_method(cls,fun):
    setattr(cls,fun.__name__,fun)
    return fun

  def __init__(self,units,context_text_processor,target_text_processor):
    super().__init__()

    encoder = Encoder(context_text_processor,units)
    decoder = Decoder(target_text_processor,units)

    self.encoder = encoder
    self.decoder = decoder

  def call(self,inputs):
    context ,x = inputs
    context = self.encoder(context)
    logits = self.decoder(context,x)

    return logits

In [ ]:
model = Translator(UNITS,context_text_processor,target_text_processor)

In [ ]:
def masked_loss(y_true, y_pred):
    loss_fn = keras.losses.SparseCategoricalCrossentropy(
        from_logits=True, reduction='none')
    loss = loss_fn(y_true, y_pred)
    mask = tf.cast(y_true != 0, loss.dtype)
    loss *= mask
    return tf.reduce_sum(loss)/tf.reduce_sum(mask)

def masked_acc(y_true, y_pred):
    y_pred = tf.argmax(y_pred, axis=-1)
    y_pred = tf.cast(y_pred, y_true.dtype)
    matchs = tf.cast(y_true == y_pred, tf.float32)
    mask = tf.cast(y_true != 0, tf.float32)
    return tf.reduce_sum(matchs)/tf.reduce_sum(mask)

In [ ]:
model.compile(optimizer='adam',
              loss=masked_loss,
              metrics=[masked_acc, masked_loss])

In [ ]:
history = model.fit(
    train_ds.repeat(),
    epochs=50,
    steps_per_epoch = 100,
    validation_data=val_ds,
    validation_steps = 20,
    callbacks=[keras.callbacks.EarlyStopping(patience=3)])

Epoch 1/50
100/100 [==============================] - 52s 230ms/step - loss: 5.3784 - masked_acc: 0.2391 - masked_loss: 5.3773 - val_loss: 4.4068 - val_masked_acc: 0.3277 - val_masked_loss: 4.4104
Epoch 2/50
100/100 [==============================] - 9s 93ms/step - loss: 3.9744 - masked_acc: 0.3765 - masked_loss: 3.9766 - val_loss: 3.6032 - val_masked_acc: 0.4185 - val_masked_loss: 3.6052
Epoch 3/50
100/100 [==============================] - 7s 68ms/step - loss: 3.3640 - masked_acc: 0.4573 - masked_loss: 3.3653 - val_loss: 3.1354 - val_masked_acc: 0.4803 - val_masked_loss: 3.1392
Epoch 4/50
100/100 [==============================] - 8s 80ms/step - loss: 2.9178 - masked_acc: 0.5173 - masked_loss: 2.9205 - val_loss: 2.7550 - val_masked_acc: 0.5402 - val_masked_loss: 2.7599
Epoch 5/50
100/100 [==============================] - 6s 62ms/step - loss: 2.5822 - masked_acc: 0.5695 - masked_loss: 2.5853 - val_loss: 2.3942 - val_masked_acc: 0.5947 - val_masked_loss: 2.3978
Epoch 6/50
100/100 [===